In [168]:
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import PIL
import os
from io import BytesIO 
from matplotlib import pyplot as plt
%matplotlib inline

In [46]:
train = pd.read_csv('data/train.csv')
train['Image'] = train['Image_Label'].map(lambda x: x.split('.')[0])
train['Label'] = train['Image_Label'].map(lambda x: x.split('_')[1])
train2 = pd.DataFrame({'Image':train['Image'][::4]})
train2['Fish_mask'] = train['EncodedPixels'][::4].values
train2['Flower_mask'] = train['EncodedPixels'][1::4].values
train2['Gravel_mask'] = train['EncodedPixels'][2::4].values
train2['Sugar_mask'] = train['EncodedPixels'][3::4].values
train2.set_index('Image',inplace=True,drop=True)
train2.fillna('0',inplace=True); train2.head()
train2[['Fish','Flower','Gravel','Sugar']] = (train2[['Fish_mask','Flower_mask','Gravel_mask','Sugar_mask']]!='0').astype('int8')
train = train2

In [47]:
train.head()

,Fish_mask,Flower_mask,Gravel_mask,Sugar_mask,Fish,Flower,Gravel,Sugar
Image,,,,,,,,
0011165,264918 937 266318 937 267718 937 269118 937 27...,1355565 1002 1356965 1002 1358365 1002 1359765...,0,0,1,1,0,0
002be4f,233813 878 235213 878 236613 878 238010 881 23...,1339279 519 1340679 519 1342079 519 1343479 51...,0,67495 350 68895 350 70295 350 71695 350 73095 ...,1,1,0,1
0031ae9,3510 690 4910 690 6310 690 7710 690 9110 690 1...,2047 703 3447 703 4847 703 6247 703 7647 703 9...,0,658170 388 659570 388 660970 388 662370 388 66...,1,1,0,1
0035239,0,100812 462 102212 462 103612 462 105012 462 10...,65400 380 66800 380 68200 380 69600 380 71000 ...,0,0,1,1,0
003994e,2367966 18 2367985 2 2367993 8 2368002 62 2369...,0,353317 416 354717 416 356117 416 357517 416 35...,28011 489 29411 489 30811 489 32211 489 33611 ...,1,0,1,1


In [169]:
def get_mask(train, idx, name):
        
    y = train.iloc[idx][name + '_mask']
        
    if type(y) == str:
        y = list(map(int, y.split()))
    else:
        y = [0, 0]
                
    mask = np.zeros(1400*2100)
                
    for index, lenght in zip(y[0::2], y[1::2]):
        index -= 1
        mask[index: index + lenght] = 1
        
    return mask.reshape((2100, 1400)).T.astype(np.bool)

In [170]:
def resize_and_crop(pilimg, scale=0.5, final_height=None):
    w = pilimg.size[0]
    h = pilimg.size[1]
    newW = int(w * scale)
    newH = int(h * scale)

    if not final_height:
        diff = 0
    else:
        diff = newH - final_height

    img = pilimg.resize((newW, newH))
    img_cropped = img.crop((0, diff // 2, newW, newH - diff // 2))
    return img_cropped

In [173]:
def Downscale(img_name, index, name, sc = 0.5):
    img = Image.open("data/train_images/" + img_name + '.jpg')
    mask = get_mask(train, index, name)
    mask  = Image.fromarray(mask)
    
    img_crop = resize_and_crop(img, scale = sc)
    mask_crop = resize_and_crop(mask, scale = sc)
    
    try:
        os.mkdir('data/Downscaled/'+ name + '_image')
        os.mkdir('data/Downscaled/'+ name + '_mask')
    except OSError:
        pass
    
    img_crop.save('data/Downscaled/'+ name + '_image/' + img_name + '.jpg')
    mask_crop.save('data/Downscaled/' + name + '_mask/' + img_name + '_mask.jpg') 

In [174]:
def get_dataset(name):
    print(name + ' dataset')
    with tqdm(total=len(train), unit='img') as pbar:
        for index in range(len(train)):
            if (train.iloc[index][name] == 1):
                img_name = train.index[index]
                Downscale(img_name, index, name)
            pbar.update(1)

In [175]:
get_dataset('Fish')

Fish dataset


100%|█████████████████████████████████████████████████████████████████████████████| 5546/5546 [08:27<00:00, 10.93img/s]


In [182]:
get_dataset('Sugar')

Sugar dataset


100%|█████████████████████████████████████████████████████████████████████████████| 5546/5546 [11:50<00:00,  7.81img/s]


In [183]:
get_dataset('Gravel')

Gravel dataset


100%|█████████████████████████████████████████████████████████████████████████████| 5546/5546 [07:46<00:00, 11.90img/s]


In [184]:
get_dataset('Flower')

Flower dataset


100%|█████████████████████████████████████████████████████████████████████████████| 5546/5546 [05:59<00:00, 15.43img/s]
